<a href="https://colab.research.google.com/github/Viktory4121/MM/blob/main/%D0%9B%D0%A03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Лабораторная работа №3.
#### Клеточные конечные автоматы.
##### Вариант 11.

In [58]:
import numpy as np
import pandas as pd
import random
import copy
import math
import itertools as it
# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

#### Входные параметры:

In [64]:
t = [i for i in range(1, 11)]
cell_auto0 = [[0 for j in range(10)] for i in range(10)]
cell_auto0[5][6] = 1
cell_auto0[9][2] = 1
cell_auto0[0][3] = 1
cell_auto0[4][4] = 1
cell_auto0[2][4] = 1
cell_auto0[3][0] = 1
cell_auto0[2][8] = 1
cell_auto0[3][5] = 1
cell_auto0[2][5] = 1
cell_auto0[3][8] = 1
cell_auto0 = pd.DataFrame(cell_auto0)
cell_auto0.columns = [i for i in range(10)]
cell_auto0.index = [i for i in range(10)]

#### Генерация основной части таблицы состояний:

In [65]:
def generation_table_states():
  table = pd.DataFrame(it.product([0,1], repeat=5))
  table[5] = [0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0]
  table.columns = [i for i in range(6)]
  table.index = [i for i in range(32)]
  return table

#### Функция проверки клетки на возбуждённое состояние:

In [92]:
def generate_cell_auto(table, avt1):
  result = [[0 for j in range(10)] for i in range(10)]

  for j in range(10):
    for k in range(10):
      #проверка на границы
      if (j == 0):
        c1 = 0
      else: c1 = avt1[j-1][k]
      if (k == 0):
        c3 = 0
      else: c3 = avt1[j][k-1]
      if (j == 9):
        c2 = 0
      else: c2 = avt1[j+1][k]
      if (k == 9):
        c4 = 0
      else: c4 = avt1[j][k+1]

      for i in range(32):
        if(table[i][0] == avt1[j][k] & table[i][1] == c1 & table[i][2] == c2 & table[i][3] == c3 & table[i][4] == c4):
          result[j][k] = 1
          break
  return result

#### Работа с таблицей состояний:

In [93]:
table_states = generation_table_states()

print('Таблица состояний для клеточного автомата:')
data = pd.DataFrame(table_states.copy())
data.columns = ['x(t-1;c1;c2)', 'x(t-1;c1-1;c2)', 'x(t-1;c1+1;c2)', 'x(t-1;c1;c2-1)', 'x(t-1;c1;c2+1)', 'x(t;c1;c2)']
print(data)
print('-------------------------------------------------------------------')

print('\nКлеточный автомат в момент времени t = 0:')
print(cell_auto0)
print('-------------------------------------------------------------------')

table_old = cell_auto0
for i in t:
  print('Клеточный автомат в момент времени t = ' + str(i))
  table_new = generate_cell_auto(np.array(table_states), table_old)
  table_old = table_new
  print_table = pd.DataFrame(table_new)
  print(print_table)
  print('-------------------------------------------------------------------')

Таблица состояний для клеточного автомата:
    x(t-1;c1;c2)  x(t-1;c1-1;c2)  x(t-1;c1+1;c2)  x(t-1;c1;c2-1)  \
0              0               0               0               0   
1              0               0               0               0   
2              0               0               0               1   
3              0               0               0               1   
4              0               0               1               0   
5              0               0               1               0   
6              0               0               1               1   
7              0               0               1               1   
8              0               1               0               0   
9              0               1               0               0   
10             0               1               0               1   
11             0               1               0               1   
12             0               1               1               0   
13   